# Pipeline Analysis

## Bias-Correction
1. Average-combine Bias frames
  - zerocombine:
    - input: list of bias images
    - output: name of combined bias frame
    - combine: average
    - ccdtype: (none)
    - gain: (gain)
    - rdnoise: (readoutnoise)
2. Subtract combination from Flat frames
  - ccdproc:
    - images: list of flat images
    - output: list of biascorrected flat images
    - ccdtype: (none)
    - everything except zerocor set to "No"
    - zero: combined bias frame
3. Subtract combination from Science frames
  - ccdproc:
    - images: list of science images
    - output: list of biascorrected science images
    - ccdtype: (none)
    - everything except zerocor set to "No"
    - zero: combined bias frame

## Flat-Correction
1. Median-combine Flat frames for every Filter separately
  - flatcombine:
    - input: list of biascorrencted flat images
    - output: combined flat image
    - combine: median
    - ccdtype: (none)
    - process: No
    - subsets: No
    - gain: (gain)
    - rdnoise: (readoutnoise)
2. Find the Mean counts in every combined Flat frame
  - imsatistics:
    - images: combined flat image
    - fields: mean
3. Divide every combined Flat frame with its Median creating the Normalized Flat frame
  - imarith:
    - operand1: combined flat image
    - op: /
    - operand2: mean
    - result: normalized flat image
4. Divide every Science image with the corresponding (for its Filter) Normalized Flat image
  - imarith:
    - operand1: list os science images in that filter
    - op: /
    - operand2: normalized flat image
    - result: list of flatcorrected science images

## Aligning a set of images
1. Pick master image (A)
2. Display image
  - ds9&
  - display A
3. Log Reference Stars on A
  - imexamine:
    - input: A
    - logfile: name of logfile
    - keeplog: Yes
4. Using the "a" keystroke in the image mark 5-10 isolated, well-defined stars
5. Align the images
  - imalign:
    - input: list with A as first image
    - reference: A
    - coords: log from imexamine
    - output: list with the shifted images names

## Combining Images
1. Median combine images
  - imcombine:
    - input: list of science images to combine
    - output: combination of the images
    - combine: median
    - gain: (gain)
    - rdnoise: (readoutnoise)

## Adding WCS to images
1. Use solve-field to add WCS to images:
```
solve-field --ra=(approximate R.A. of our field) --dec=(approximate R.A. of our field) -5 (radius to search in deg (skinakas=0.5 good choice)) --out=(output image) -t=(order of fitted polynomial (I use 2)) --sigma=(3*backgroundstd) (input image)
```

## Allstar!!!
1. Examine image to find the FWHM and the background STD
  - ds9&
  - display (image)
  - imexamine:
    - input: image
  - Press a key inside some stars and m key inside some background areas
  - FWHM=MOFFAT (from stars)
  - BGSTD=STD (from background)
2. Detect point-sources in image
  - daofind:
    - image: image that will be allstarred
    - output: output pointsource catalog
    - datapars:
      - scale: 1
      - fwhmpsf: FWHM
      - emission: Yes
      - sigma: BGSTD
      - noise: poisson
      - readnoise: (readoutnoise)
      - epadu: (gain)
      - itime: (exposuretime)
    - findpars:
      - threshold: 4 or 3
    - boundary: nearest
    - constant: 0
3. Photometry on the point sources
  - phot:
    - image: image that will be allstarred
    - coords: pointsource catalog
    - output: output photometry catalog
    - centerpars:
      - calgorithm: centroid
      - cbox: 5
    - fitskypars:
      - salgorithm: mode
      - annulus: 3\*FWHM±3
      - dannulus: 8±2
      - skyvalue: 0
    - photpars:
      - weighting: constant
      - apertures: 2\*FWHM+(5±3)
4. Select the stars to be used to create the model PSF
  - ds9&
  - display (image)
  - pstselect:
    - image: image that will be allstarred
    - phtofile: photometry catalog
    - pstfile: output psf star list catalog
    - maxnpsf: 90
    - daopars:
      - function: auto
      - varorder: 2
      - nclean: 0
      - saturated: No
      - matchrad: 3±1
      - psfrad: 2\*FWHM+(3±2)
      - fitrad: 2\*FWHM±2
      - recenter: Yes
      - fitsky: Yes
      - groupsky: Yes if crowded, No otherwise
      - sannulus: 3\*FWHM±3
      - wsannulus: 8±2
      - maxiter: 100
    - interactive: Yes
  - Press "n" key on ds9 window then go on plot, see if star is well-defined then press "a" if good or "d" if bad. Repeat until the terminal says you reached limit. Press "q" on ds9
5. Create the psf model
  - psf:
    - image: image that will be allstarred
    - photfile: photometry catalog
    - pstfile: psf star list catalog
    - psfimage: output psf model
    - opstfile: output list of good PSF stars
    - groupfile: output PSF star group file
    - matchbyid: No
    - interactive: No
6. Allstar
  - allstar:
    - image: image that will be allstarred
    - photfile: photometry catalog
    - psfimage: output psf model
    - allstarfile: output photometry file of removed stars
    - rejfile: output file for rejected stars
    - subimage: allstarsubtracted image
7. Reruns
  - At least two more times execute:
    - Step 3 on allstarsubtracted
    - pfmerge:
      - inphotfiles: (latest allstarfile, latest photometry of subtracted image)
      - outphotfile: output merged catalog
    - Step 6 but now photfile is the merged catalog
8. Run Step 7 as many times as needed

## Scaling
1. Dump the photometry catalogs into ascii format
  - tdump:
    - catalogs: the catalogs for the narrow and for the broad images producedd at Step 3 of Allstar Section
    - datafile: files to dump the info at
    - columns: the columns containing the X center, Y center and FLUX of stars
2. Using the catalogs
  - topcat:
    - Load the ascii catalogs
    - 2D Cartesian match of coordinates metween catalogs
    - New column containing the ratio of fluxes
    - Median of column is the scaling factor